In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
train = pd.read_csv('./data/airline_satisfaction_train.csv', index_col=0).set_index('id').dropna().assign(train=True)
test = pd.read_csv('./data/airline_satisfaction_test.csv', index_col=0).set_index('id').dropna().assign(train=False)
df = pd.concat([train, test])
with pd.option_context('display.max_columns', None):
    display(df)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction,train
id,,,,,,,,,,,,,,,,,,,,,,,,
70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied,True
5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied,True
110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0,satisfied,True
24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied,True
119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0,satisfied,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78463,Male,disloyal Customer,34,Business travel,Business,526,3,3,3,1,4,3,4,4,3,2,4,4,5,4,0,0.0,neutral or dissatisfied,False
71167,Male,Loyal Customer,23,Business travel,Business,646,4,4,4,4,4,4,4,4,4,5,5,5,5,4,0,0.0,satisfied,False
37675,Female,Loyal Customer,17,Personal Travel,Eco,828,2,5,1,5,2,1,2,2,4,3,4,5,4,2,0,0.0,neutral or dissatisfied,False


In [10]:
categorical = [
    'Gender',
    'Customer Type',
    'Type of Travel',
    'Class',
]

scores = [
    'Inflight wifi service',
    'Departure/Arrival time convenient',
    'Ease of Online booking',
    'Gate location',
    'Food and drink',
    'Online boarding',
    'Seat comfort',
    'Inflight entertainment',
    'On-board service',
    'Leg room service',
    'Baggage handling',
    'Checkin service',
    'Inflight service',
    'Cleanliness'
]

numeric = {
    'Age': [-0.01, 20, 40, 60, np.inf],
    'Flight Distance': [-0.01, 1000, 2000, 3000, 4000, np.inf],
    'Departure Delay in Minutes': [-0.01, 60, 4*60, np.inf],
    'Arrival Delay in Minutes': [-0.01, 60, 4*60, np.inf],
}

In [11]:
df = df[categorical + list(numeric.keys()) + scores + ['satisfaction']]

for col in categorical:
    print(col)
    binarized = pd.get_dummies(df[col], prefix=col)
    df = df.join(binarized).drop(columns=[col])

df['satisfaction'] = (df['satisfaction'] == 'satisfied')
df = df.rename(columns={'satisfaction': 'satisfaction_satisfied'})
df = df.astype(float)
with pd.option_context('display.max_columns', None):
    display(df)

Gender
Customer Type
Type of Travel
Class


,Age,Flight Distance,Departure Delay in Minutes,Arrival Delay in Minutes,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,satisfaction_satisfied,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70172,13.0,460.0,25.0,18.0,3.0,4.0,3.0,1.0,5.0,3.0,5.0,5.0,4.0,3.0,4.0,4.0,5.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
5047,25.0,235.0,1.0,6.0,3.0,2.0,3.0,3.0,1.0,3.0,1.0,1.0,1.0,5.0,3.0,1.0,4.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
110028,26.0,1142.0,0.0,0.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,4.0,3.0,4.0,4.0,4.0,5.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
24026,25.0,562.0,11.0,9.0,2.0,5.0,5.0,5.0,2.0,2.0,2.0,2.0,2.0,5.0,3.0,1.0,4.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
119299,61.0,214.0,0.0,0.0,3.0,3.0,3.0,3.0,4.0,5.0,5.0,3.0,3.0,4.0,4.0,3.0,3.0,3.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78463,34.0,526.0,0.0,0.0,3.0,3.0,3.0,1.0,4.0,3.0,4.0,4.0,3.0,2.0,4.0,4.0,5.0,4.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
71167,23.0,646.0,0.0,0.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,5.0,5.0,4.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
37675,17.0,828.0,0.0,0.0,2.0,5.0,1.0,5.0,2.0,1.0,2.0,2.0,4.0,3.0,4.0,5.0,4.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [12]:
airline_X = df.drop(columns=['satisfaction_satisfied'])
airline_y = df['satisfaction_satisfied']
airline_X_train, airline_X_test, airline_y_train, airline_y_test = train_test_split(
    airline_X,
    airline_y,
    train_size=700,
    test_size=300,
    random_state=0,
    stratify=airline_y
)
airline_X_train.to_csv('airline_X_train.csv', index=False)
airline_X_test.to_csv('airline_X_test.csv', index=False)
airline_y_train.to_csv('airline_y_train.csv', index=False)
airline_y_test.to_csv('airline_y_test.csv', index=False)

In [13]:
df_bin = df.copy()


for col in scores:
    print(col)
    binarized = (df[[col]] >= 4).rename(columns={col: f'{col}_ge_4'})
    df_bin = df_bin.join(binarized).drop(columns=[col])

for col, bins in numeric.items():
    print(col)
    binarized = pd.get_dummies(pd.cut(df[col], bins=bins), prefix=f'{col}')
    df_bin = df_bin.join(binarized).drop(columns=[col])

df_bin = df_bin.astype(int)
with pd.option_context('display.max_columns', None):
    display(df_bin)


Inflight wifi service
Departure/Arrival time convenient
Ease of Online booking
Gate location
Food and drink
Online boarding
Seat comfort
Inflight entertainment
On-board service
Leg room service
Baggage handling
Checkin service
Inflight service
Cleanliness
Age
Flight Distance
Departure Delay in Minutes
Arrival Delay in Minutes


,satisfaction_satisfied,Gender_Female,Gender_Male,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus,Inflight wifi service_ge_4,Departure/Arrival time convenient_ge_4,Ease of Online booking_ge_4,Gate location_ge_4,Food and drink_ge_4,Online boarding_ge_4,Seat comfort_ge_4,Inflight entertainment_ge_4,On-board service_ge_4,Leg room service_ge_4,Baggage handling_ge_4,Checkin service_ge_4,Inflight service_ge_4,Cleanliness_ge_4,"Age_(-0.01, 20.0]","Age_(20.0, 40.0]","Age_(40.0, 60.0]","Age_(60.0, inf]","Flight Distance_(-0.01, 1000.0]","Flight Distance_(1000.0, 2000.0]","Flight Distance_(2000.0, 3000.0]","Flight Distance_(3000.0, 4000.0]","Flight Distance_(4000.0, inf]","Departure Delay in Minutes_(-0.01, 60.0]","Departure Delay in Minutes_(60.0, 240.0]","Departure Delay in Minutes_(240.0, inf]","Arrival Delay in Minutes_(-0.01, 60.0]","Arrival Delay in Minutes_(60.0, 240.0]","Arrival Delay in Minutes_(240.0, inf]"
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
70172,0,0,1,1,0,0,1,0,0,1,0,1,0,0,1,0,1,1,1,0,1,1,1,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0
5047,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
110028,1,1,0,1,0,1,0,1,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0
24026,0,1,0,1,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
119299,1,0,1,1,0,1,0,1,0,0,0,0,0,0,1,1,1,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78463,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,1,1,0,0,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
71167,1,0,1,1,0,1,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0
37675,0,1,0,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,1,1,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0


In [14]:
airline_bin_X = df_bin.drop(columns=['satisfaction_satisfied'])
airline_bin_y = df_bin['satisfaction_satisfied']
airline_bin_X_train, airline_bin_X_test, airline_bin_y_train, airline_bin_y_test = train_test_split(
    airline_bin_X,
    airline_bin_y,
    train_size=700,
    test_size=300,
    random_state=0,
    stratify=airline_bin_y
)
airline_bin_X_train.to_csv('airline_bin_X_train.csv', index=False)
airline_bin_X_test.to_csv('airline_bin_X_test.csv', index=False)
airline_bin_y_train.to_csv('airline_bin_y_train.csv', index=False)
airline_bin_y_test.to_csv('airline_bin_y_test.csv', index=False)